In [1]:
# Load libraries
from sklearn.ensemble import AdaBoostClassifier
# Import Counter
from collections import Counter
# Import LabelEncoder
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
# Import Naive Bayes Classifier
from sklearn.naive_bayes import GaussianNB
# Handling data imbalancing
from imblearn.over_sampling import SMOTE
# Import pandas for reading data
import pandas as pd
# Import train_test_split function
from sklearn.model_selection import train_test_split
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

ModuleNotFoundError: No module named 'imblearn'

In [ ]:
# Dealing with segmented data
#"""select count(1) from "AI_LAGORTA"."CAMP_AUG15_Y_CBM1""""

from sqlalchemy import create_engine
import cx_Oracle


host='192.168.113.80'
port=1521
sid='bbidb'
user='system'
password='oraclebbi'
sid = cx_Oracle.makedsn(host, port, sid=sid)

def oracle_connect(host=host, sid=sid, user=user, password=password):
    cstr = 'oracle://{user}:{password}@{sid}/'.format (
        user = user ,
        password = password ,
        sid = sid
    )

    engine = create_engine (
        cstr ,
        convert_unicode = False ,
        pool_recycle = 10 ,
        pool_size = 50 ,
        echo =False
    )

    return engine.connect()

In [26]:
# Dealing with segmented data
#"""select count(1) from "AI_LAGORTA"."CAMP_AUG15_Y_CBM1""""

from sqlalchemy import create_engine
import cx_Oracle


host='192.168.113.80'
port=1521
sid='bbidb'
user='system'
password='oraclebbi'
sid = cx_Oracle.makedsn(host, port, sid=sid)

def oracle_connect(host=host, sid=sid, user=user, password=password):
    cstr = 'oracle://{user}:{password}@{sid}/'.format (
        user = user ,
        password = password ,
        sid = sid
    )

    engine = create_engine (
        cstr ,
        convert_unicode = False ,
        pool_recycle = 10 ,
        pool_size = 50 ,
        echo =False
    )

    return engine.connect()

In [27]:
# retrieve database table
con = oracle_connect()
df = pd.read_sql_query('SELECT * FROM AI_LAGORTA.CAMP_AUG15_Y_CBM2', con = con)

Exception during reset or similar
Traceback (most recent call last):
  File "C:\Users\bassam.abdelmohsen\AppData\Local\Continuum\anaconda3\lib\site-packages\sqlalchemy\pool\base.py", line 680, in _finalize_fairy
    fairy._reset(pool)
  File "C:\Users\bassam.abdelmohsen\AppData\Local\Continuum\anaconda3\lib\site-packages\sqlalchemy\pool\base.py", line 867, in _reset
    pool._dialect.do_rollback(self)
  File "C:\Users\bassam.abdelmohsen\AppData\Local\Continuum\anaconda3\lib\site-packages\sqlalchemy\engine\default.py", line 530, in do_rollback
    dbapi_connection.rollback()
cx_Oracle.OperationalError: ORA-03135: connection lost contact
Process ID: 7428
Session ID: 928 Serial number: 23442


In [28]:
# handling ordinal data
v_customer_value_enc = OrdinalEncoder ( categories = [ [ 'LVC' , 'HVC' ] ] )
nationality_group_enc = OrdinalEncoder ( categories = [ [ 'Low' , 'Med' , "High" ] ] )
df [ 'v_customer_value$' ] = v_customer_value_enc.fit_transform (df [ 'v_customer_value$' ].values.reshape ( -1 , 1 ) )
df [ 'nationality_group' ] = nationality_group_enc.fit_transform (df [ 'nationality_group' ].values.reshape ( -1 , 1 ) )
# separate predictors and target
X = df.drop(['pk_msisdn_id', 'target_all'], axis=1)
y = df['target_all'].values

In [29]:
# split dataset into training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify = y) # 70% training and 30% test


In [30]:
len(y_train), len(y_test)

(16672, 7146)

In [31]:
# Training class count
import numpy as np

unique, counts = np.unique(y_train, return_counts=True)
percent = (counts / len(y_train)) * 100
print(np.asarray((unique, percent)).T)

[['0' 96.42514395393475]
 ['1' 3.574856046065259]]


In [32]:
# Testing class count
unique, counts = np.unique(y_test, return_counts=True)
percent = counts 
print(np.asarray((unique, percent)).T)

[['0' 6891]
 ['1' 255]]


In [33]:
# Handling data imbalancing
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)
print('Original dataset shape %s' % Counter(y_train))
print('Resampled dataset shape %s' % Counter(y_res))

Original dataset shape Counter({'0': 16076, '1': 596})
Resampled dataset shape Counter({'0': 16076, '1': 16076})


In [34]:
# create classifier
naivebayes = GaussianNB()
# Create adaboost classifer object
ad_boost_c =AdaBoostClassifier(n_estimators=100, base_estimator=naivebayes ,learning_rate=1)
# Train Adaboost Classifer
model = ad_boost_c.fit(X_res, y_res)

In [35]:
# Training Data Metrics
target_names = ['class 0', 'class 1']
print(metrics.classification_report(y_res, model.predict(X_res), target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.55      0.62      0.58     16076
     class 1       0.56      0.49      0.52     16076

    accuracy                           0.55     32152
   macro avg       0.56      0.55      0.55     32152
weighted avg       0.56      0.55      0.55     32152



In [36]:
# Testing Data Metrics
target_names = ['class 0', 'class 1']
print(metrics.classification_report(y_test, model.predict(X_test), target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.96      0.62      0.76      6891
     class 1       0.04      0.38      0.07       255

    accuracy                           0.61      7146
   macro avg       0.50      0.50      0.41      7146
weighted avg       0.93      0.61      0.73      7146

